In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Function to extract text from PDF
def extract_text_from_pdf(file):
    pdf = PdfReader(file)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [ ]:
# Function to metric applications based on job description


def metric_applications(job_description, applications):
    documents = [job_description] + applications  # Combine job description with resumes
    vectorizer = TfidfVectorizer().fit_transform(documents)  # Convert text to numerical vectors
    vectors = vectorizer.toarray()
    
    job_description_vector = vectors[0]  # First vector is the job description
    application_vectors = vectors[1:]  # Remaining vectors are resumes
    
    cosine_similarities = cosine_similarity([job_description_vector], application_vectors).flatten()
    
    return cosine_similarities  # Return similarity scores for all resumes


In [ ]:
# Streamlit app
st.title("AI Resume Screening & Candidate Ranking System")
# Job description input
st.header("Job Description")
job_description= st.text_area ("Enter the job description")

In [ ]:
# File uploader
st.header("Upload Resumes")
uploaded_files = st.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

In [ ]:
if uploaded_files and job_description:
    st.header("Ranking Resumes")
    
    applications = []
    for file in uploaded_files:
        text = extract_text_from_pdf(file)
        applications.append(text)

    # Rank applications
    metrics = metric_applications(job_description, applications)

    # Display metrics
    results = pd.DataFrame({"Resume": [file.name for file in uploaded_files], "Score": metric })
    results = results.sort_values(by="Score", ascending=False)
    
    st.write(results)